In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
# parameters
learning_rate = 0.001
epochs = 50
batch_size = 100
drop_out = 0.3

In [15]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [16]:
# CIFAR10 데이타셋
training_data = datasets.CIFAR10(root='CIFAR10_data/',
                          train=True,
                          transform=transform,
                          download=True)

test_data = datasets.CIFAR10(root='CIFAR10_data/',
                         train=False,
                         transform=transform,
                         download=True)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

In [18]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) # input, output, kerner_size=filter_size, stride, padding
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.pool = nn.MaxPool2d(2, 2) # kernel_size, stride
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  # 컨볼루션에 데이터셋을 통과시킴
  def forward(self, out):
    out = self.pool(nn.functional.relu(self.conv1(out)))
    out = self.pool(nn.functional.relu(self.conv2(out)))
    out = torch.flatten(out, 1) # 배치를 제외한 모든 차원을 평탄화
    out = nn.functional.relu(self.fc1(out))
    out = nn.functional.relu(self.fc2(out))
    out = self.fc3(out)
    return out

In [19]:
model = CNN().to(device)

In [20]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
total_batch = len(train_dataloader)
print('Learning started. It takes sometime.')

for epoch in range(epochs):
  avg_loss = 0                                  # 비용 초기값

  for i, data in enumerate(train_dataloader):
    inputs, labels = data

    inputs = inputs.to(device)                  # inputs -> 디바이스로 옮김
    labels = labels.to(device)                  # labels -> 디바이스로 옮김

    # 변화도(Gradient) 매개변수를 0으로 만듦
    optimizer.zero_grad()

    # 순전파 + 역전파 + 최적화
    outputs = model(inputs)                     # 가설 설정
    loss = criterion(outputs, labels)           # 로스 설정
    loss.backward()                             # 역전파
    optimizer.step()                            # 학습을 통해 계산한 weight, bias, gradient를 최적화 함수(optimizer)에 반영

    avg_loss += loss / total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_loss))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 1.70231247
[Epoch:    2] cost = 1.42559171
[Epoch:    3] cost = 1.31290829
[Epoch:    4] cost = 1.22338331
[Epoch:    5] cost = 1.15137088
[Epoch:    6] cost = 1.09904695
[Epoch:    7] cost = 1.04773724
[Epoch:    8] cost = 1.00823069
[Epoch:    9] cost = 0.967262745
[Epoch:   10] cost = 0.936877072
[Epoch:   11] cost = 0.903025806
[Epoch:   12] cost = 0.872737348
[Epoch:   13] cost = 0.842510998
[Epoch:   14] cost = 0.815199733
[Epoch:   15] cost = 0.792832971
[Epoch:   16] cost = 0.76855129
[Epoch:   17] cost = 0.747142255
[Epoch:   18] cost = 0.721577704
[Epoch:   19] cost = 0.704282045
[Epoch:   20] cost = 0.682630956
[Epoch:   21] cost = 0.66617924
[Epoch:   22] cost = 0.650818825
[Epoch:   23] cost = 0.626951694
[Epoch:   24] cost = 0.615055799
[Epoch:   25] cost = 0.601026893
[Epoch:   26] cost = 0.580083013
[Epoch:   27] cost = 0.567441344
[Epoch:   28] cost = 0.55329138
[Epoch:   29] cost = 0.542483449
[Epoch:   30] cos

In [22]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data

        images = images.to(device)
        labels = labels.to(device)

        # 신경망에 이미지를 통과시켜 출력을 계산
        outputs = model(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 61 %
